In [1]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~16.04.1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /home/gabe/Envs/h2o/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpS67leV
  JVM stdout: /tmp/tmpS67leV/h2o_gabe_started_from_python.out
  JVM stderr: /tmp/tmpS67leV/h2o_gabe_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Argentina/Buenos_Aires
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_gabe_08cgbx
H2O cluster total nodes:,1
H2O cluster free memory:,3.431 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("./1audience_dummy.csv")
test = h2o.import_file("./1audience_dummy.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
# Identify predictors and response
x = train.columns
y = "er_label"
x.remove(y)
x.remove('C1')
x.remove('device_id')
x.remove('key_evs_count')

In [6]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [12]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
GLM_grid_1_AutoML_20191120_092216_model_1,0.665784,0.197202,0.390206,0.221154,0.0489092
DRF_1_AutoML_20191120_092216,0.614378,0.198535,0.357733,0.221079,0.0488757
XGBoost_2_AutoML_20191120_092216,0.613049,0.203628,0.350501,0.225346,0.0507807
StackedEnsemble_BestOfFamily_AutoML_20191120_092216,0.610535,0.199081,0.323758,0.222296,0.0494153
StackedEnsemble_AllModels_AutoML_20191120_092216,0.606626,0.199204,0.323758,0.222318,0.0494252
XGBoost_3_AutoML_20191120_092216,0.585882,0.205528,0.354295,0.226218,0.0511745
XGBoost_1_AutoML_20191120_092216,0.58437,0.20481,0.350501,0.225795,0.0509833
GBM_3_AutoML_20191120_092216,0.561864,0.20962,0.358089,0.227327,0.0516774
GBM_5_AutoML_20191120_092216,0.560767,0.205551,0.350501,0.225562,0.0508784
GBM_2_AutoML_20191120_092216,0.553582,0.209972,0.350587,0.227596,0.0518


In [9]:
# The leader model is stored here
aml.leader

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_grid_1_AutoML_20191120_092216_model_1


GLM Model: summary


,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,binomial,logit,Ridge ( lambda = 0.009625 ),"nlambda = 30, lambda.max = 4.7102, lambda.min = 0.009625, lambda.1se = 4.7102",7,7,31,automl_training_py_1_sid_be3c




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.0469883143138
RMSE: 0.216767881186
LogLoss: 0.191328964566
Null degrees of freedom: 998
Residual degrees of freedom: 991
Null deviance: 425.851195682
Residual deviance: 382.275271203
AIC: 398.275271203
AUC: 0.695464175655
pr_auc: 0.234982936724
Gini: 0.39092835131

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0969790650539: 


,0,1,Error,Rate
0,916.0,28.0,0.0297,(28.0/944.0)
1,41.0,14.0,0.7455,(41.0/55.0)
Total,957.0,42.0,0.0691,(69.0/999.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0110110,0.3719222,8.2561983,8.2561983,0.4545455,0.5619340,0.4545455,0.5619340,0.0909091,0.0909091,725.6198347,725.6198347
,2,0.0200200,0.1916470,4.0363636,6.3572727,0.2222222,0.2802770,0.35,0.4351883,0.0363636,0.1272727,303.6363636,535.7272727
,3,0.0300300,0.1213470,5.4490909,6.0545455,0.3,0.1541271,0.3333333,0.3415012,0.0545455,0.1818182,444.9090909,505.4545455
,4,0.0400400,0.0889892,3.6327273,5.4490909,0.2,0.1067982,0.3,0.2828255,0.0363636,0.2181818,263.2727273,444.9090909
,5,0.0510511,0.0818795,3.3024793,4.9860963,0.1818182,0.0859684,0.2745098,0.2403661,0.0363636,0.2545455,230.2479339,398.6096257
,6,0.1011011,0.0598589,1.4530909,3.2370837,0.08,0.0664378,0.1782178,0.1542630,0.0727273,0.3272727,45.3090909,223.7083708
,7,0.1661662,0.0536525,2.2355245,2.8449069,0.1230769,0.0548938,0.1566265,0.1153534,0.1454545,0.4727273,123.5524476,184.4906900
,8,0.2192192,0.0499522,1.0281304,2.4052304,0.0566038,0.0501598,0.1324201,0.0995759,0.0545455,0.5272727,2.8130360,140.5230386
,9,0.3123123,0.0472047,1.1718475,2.0375874,0.0645161,0.0480387,0.1121795,0.0842139,0.1090909,0.6363636,17.1847507,103.7587413
,10,0.4694695,0.0445364,0.2313839,1.4329521,0.0127389,0.0446263,0.0788913,0.0709618,0.0363636,0.6727273,-76.8616097,43.2952123




Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9389397,0.0089369,0.94,0.925,0.95,0.94,0.9396985
auc,0.7074272,0.1206468,0.7318421,0.4989716,0.8115234,0.7389963,0.7558027
err,0.0610603,0.0089369,0.06,0.075,0.05,0.06,0.0603015
err_count,12.2,1.7888544,12.0,15.0,10.0,12.0,12.0
f0point5,0.3624667,0.0825006,0.3571429,0.2702703,0.3125,0.4878049,0.3846154
---,---,---,---,---,---,---,---
r2,0.0721133,0.0423934,0.1216414,0.0065651,0.0658376,0.0745645,0.0919581
recall,0.2568531,0.0619590,0.3,0.1538462,0.25,0.3076923,0.2727273
residual_deviance,77.82422,13.483359,69.48663,95.34003,61.723724,87.10343,75.46728
rmse,0.2188578,0.0231211,0.2042597,0.2457153,0.1893986,0.2371568,0.2177585



Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2019-11-20 09:22:37,0.000 sec,2,.47E1,8,0.4233019,nan,0.4239232,0.0276147
,2019-11-20 09:22:37,0.003 sec,4,.29E1,8,0.4216152,nan,0.4221956,0.0280600
,2019-11-20 09:22:37,0.005 sec,6,.18E1,8,0.4191302,nan,0.4200045,0.0286021
,2019-11-20 09:22:37,0.008 sec,8,.11E1,8,0.4156977,nan,0.4175372,0.0291025
,2019-11-20 09:22:37,0.010 sec,10,.7E0,8,0.4112526,nan,0.4147429,0.0294949
,2019-11-20 09:22:37,0.012 sec,12,.44E0,8,0.4058482,nan,0.4112570,0.0298673
,2019-11-20 09:22:37,0.014 sec,14,.27E0,8,0.3999285,nan,0.4071069,0.0302464
,2019-11-20 09:22:37,0.016 sec,16,.17E0,8,0.3943277,nan,0.4029004,0.0305791
,2019-11-20 09:22:37,0.018 sec,18,.1E0,8,0.3897745,nan,0.3993783,0.0308847
,2019-11-20 09:22:37,0.019 sec,20,.65E-1,8,0.3865779,nan,0.3968939,0.0312676


In [13]:
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly
preds = aml.leader.predict(test)
aml.leader

glm prediction progress: |████████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_grid_1_AutoML_20191120_092950_model_1


GLM Model: summary


,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,binomial,logit,Ridge ( lambda = 0.009625 ),"nlambda = 30, lambda.max = 4.7102, lambda.min = 0.009625, lambda.1se = 4.7102",7,7,31,automl_training_py_1_sid_be3c




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.0469883143138
RMSE: 0.216767881186
LogLoss: 0.191328964566
Null degrees of freedom: 998
Residual degrees of freedom: 991
Null deviance: 425.851195682
Residual deviance: 382.275271203
AIC: 398.275271203
AUC: 0.695464175655
pr_auc: 0.234982936724
Gini: 0.39092835131

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0969790650539: 


,0,1,Error,Rate
0,916.0,28.0,0.0297,(28.0/944.0)
1,41.0,14.0,0.7455,(41.0/55.0)
Total,957.0,42.0,0.0691,(69.0/999.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0110110,0.3719222,8.2561983,8.2561983,0.4545455,0.5619340,0.4545455,0.5619340,0.0909091,0.0909091,725.6198347,725.6198347
,2,0.0200200,0.1916470,4.0363636,6.3572727,0.2222222,0.2802770,0.35,0.4351883,0.0363636,0.1272727,303.6363636,535.7272727
,3,0.0300300,0.1213470,5.4490909,6.0545455,0.3,0.1541271,0.3333333,0.3415012,0.0545455,0.1818182,444.9090909,505.4545455
,4,0.0400400,0.0889892,3.6327273,5.4490909,0.2,0.1067982,0.3,0.2828255,0.0363636,0.2181818,263.2727273,444.9090909
,5,0.0510511,0.0818795,3.3024793,4.9860963,0.1818182,0.0859684,0.2745098,0.2403661,0.0363636,0.2545455,230.2479339,398.6096257
,6,0.1011011,0.0598589,1.4530909,3.2370837,0.08,0.0664378,0.1782178,0.1542630,0.0727273,0.3272727,45.3090909,223.7083708
,7,0.1661662,0.0536525,2.2355245,2.8449069,0.1230769,0.0548938,0.1566265,0.1153534,0.1454545,0.4727273,123.5524476,184.4906900
,8,0.2192192,0.0499522,1.0281304,2.4052304,0.0566038,0.0501598,0.1324201,0.0995759,0.0545455,0.5272727,2.8130360,140.5230386
,9,0.3123123,0.0472047,1.1718475,2.0375874,0.0645161,0.0480387,0.1121795,0.0842139,0.1090909,0.6363636,17.1847507,103.7587413
,10,0.4694695,0.0445364,0.2313839,1.4329521,0.0127389,0.0446263,0.0788913,0.0709618,0.0363636,0.6727273,-76.8616097,43.2952123




Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9389397,0.0089369,0.94,0.925,0.95,0.94,0.9396985
auc,0.7074272,0.1206468,0.7318421,0.4989716,0.8115234,0.7389963,0.7558027
err,0.0610603,0.0089369,0.06,0.075,0.05,0.06,0.0603015
err_count,12.2,1.7888544,12.0,15.0,10.0,12.0,12.0
f0point5,0.3624667,0.0825006,0.3571429,0.2702703,0.3125,0.4878049,0.3846154
---,---,---,---,---,---,---,---
r2,0.0721133,0.0423934,0.1216414,0.0065651,0.0658376,0.0745645,0.0919581
recall,0.2568531,0.0619590,0.3,0.1538462,0.25,0.3076923,0.2727273
residual_deviance,77.82422,13.483359,69.48663,95.34003,61.723724,87.10343,75.46728
rmse,0.2188578,0.0231211,0.2042597,0.2457153,0.1893986,0.2371568,0.2177585



Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2019-11-20 09:30:11,0.000 sec,2,.47E1,8,0.4233019,nan,0.4239232,0.0276147
,2019-11-20 09:30:11,0.001 sec,4,.29E1,8,0.4216152,nan,0.4221956,0.0280600
,2019-11-20 09:30:11,0.002 sec,6,.18E1,8,0.4191302,nan,0.4200045,0.0286021
,2019-11-20 09:30:11,0.002 sec,8,.11E1,8,0.4156977,nan,0.4175372,0.0291025
,2019-11-20 09:30:11,0.003 sec,10,.7E0,8,0.4112526,nan,0.4147429,0.0294949
,2019-11-20 09:30:11,0.004 sec,12,.44E0,8,0.4058482,nan,0.4112570,0.0298673
,2019-11-20 09:30:11,0.004 sec,14,.27E0,8,0.3999285,nan,0.4071069,0.0302464
,2019-11-20 09:30:11,0.005 sec,16,.17E0,8,0.3943277,nan,0.4029004,0.0305791
,2019-11-20 09:30:11,0.006 sec,18,.1E0,8,0.3897745,nan,0.3993783,0.0308847
,2019-11-20 09:30:11,0.006 sec,20,.65E-1,8,0.3865779,nan,0.3968939,0.0312676
